In [1]:
import sys
sys.path.append("../")

In [23]:
statement_name = "../data/Справка_о_движении_денежных_средств_1030.pdf"
incomes = 935544.82
expences = -981949.29

# 1. Прочитать выписку

In [24]:
import numpy as np

In [25]:
from src.finman.utils.pdf import extract_text_from_pdf
from src.finman.expenses.tbank import parse_operations

In [26]:
text = extract_text_from_pdf(statement_name)
transactions_df = parse_operations(text)

In [27]:
assert np.isclose(transactions_df.loc[transactions_df['Сумма операции в валюте карты'] > 0]['Сумма операции в валюте карты'].sum(), incomes)
assert np.isclose(transactions_df.loc[transactions_df['Сумма операции в валюте карты'] < 0]['Сумма операции в валюте карты'].sum(), expences)

In [28]:
transactions_df.shape

(172, 9)

In [29]:
transactions_df.head()

,id,Дата операции,Время операции,Сумма операции в валюте карты,Валюта карты,Сумма в валюте операции,Валюта операции,Описание операции,Номер карты
0,701a707742,2024-10-30,2024-10-30 20:27:00,-4000.0,₽,-4000.0,₽,Перевод на карту другого банка,5037
1,675c171212,2024-10-30,2024-10-30 12:57:00,-2283.1,₽,-2283.1,₽,Оплата в Yandex.Lavka Moskva RUS,2475
2,2ef6d9a28b,2024-10-30,2024-10-30 11:39:00,-2730.0,₽,-2730.0,₽,Оплата в EGGSELLENT MOSKVA RUS,1665
3,dd282173e5,2024-10-29,2024-10-29 21:53:00,-900.0,₽,-900.0,₽,Внутренний перевод на договор,5427
4,dcb8f77afa,2024-10-29,2024-10-29 20:49:00,-50.0,₽,-50.0,₽,Внутренний перевод на договор,5044


# 2. Запроцессить гугол

In [30]:
from src.finman.utils.gsheet import GSheetWorker

In [31]:
sheet_id = "1GqwNPuOtQUXfltK6F4oHpx-U1BqBgiDZfC5YfDcJyto"
tbank_main_spreadsheet_name = "t-bank (main)"
gcreds_file = "../secrets/finman-433017-da308d823497.json"

In [32]:
gsw = GSheetWorker(gcreds_file)

In [33]:
gs_df = gsw.get_df(sheet_id, tbank_main_spreadsheet_name)

In [34]:
gs_df.head()

,id,Дата операции,Время операции,Сумма операции в валюте карты,Валюта карты,Сумма в валюте операции,Валюта операции,Описание операции,Номер карты,Категория,Подкатегория,Хэштэг,Долг,Комментарий
0,7711abac36,"28.10, Mon",2024-10-28 10:49,-3573.4,₽,-3573.4,₽,Оплата в vkusvill CHERNOGOLOVKA RUS,2475,Household,Groceries,None,None,None
1,27617a48a0,"27.10, Sun",2024-10-27 12:26,-1140,₽,-1140,₽,Оплата в PTICHE MOLOKO MOSCOW RUS,1665,Household,Groceries,None,None,None
2,65a4568d33,"27.10, Sun",2024-10-27 11:09,-4000,₽,-4000,₽,Операция в других кредитных организациях YANDE...,3685,Transfer,Internal,None,None,None
3,dd634878d7,"27.10, Sun",2024-10-27 8:31,-1629.1,₽,-1629.1,₽,Оплата в Yandex.Lavka Moskva RUS,2475,Food & Dining,Delivery,None,None,None
4,e31c33ea29,"26.10, Sat",2024-10-26 15:41,-95.5,₽,-95.5,₽,Оплата в BELKACAR MOSKVA RUS,5223,Transportation,Carsharing,None,None,None


In [35]:
missing_recrods = transactions_df.loc[~transactions_df['id'].isin(gs_df['id'])]

In [36]:
missing_recrods

,id,Дата операции,Время операции,Сумма операции в валюте карты,Валюта карты,Сумма в валюте операции,Валюта операции,Описание операции,Номер карты
0,701a707742,2024-10-30,2024-10-30 20:27:00,-4000.00,₽,-4000.00,₽,Перевод на карту другого банка,5037
1,675c171212,2024-10-30,2024-10-30 12:57:00,-2283.10,₽,-2283.10,₽,Оплата в Yandex.Lavka Moskva RUS,2475
2,2ef6d9a28b,2024-10-30,2024-10-30 11:39:00,-2730.00,₽,-2730.00,₽,Оплата в EGGSELLENT MOSKVA RUS,1665
3,dd282173e5,2024-10-29,2024-10-29 21:53:00,-900.00,₽,-900.00,₽,Внутренний перевод на договор,5427
4,dcb8f77afa,2024-10-29,2024-10-29 20:49:00,-50.00,₽,-50.00,₽,Внутренний перевод на договор,5044
5,ad24a36792,2024-10-29,2024-10-29 20:48:00,2300.00,₽,2300.00,₽,Внутрибанковский перевод с договора,5158
6,f76353be25,2024-10-29,2024-10-29 20:45:00,2300.00,₽,2300.00,₽,Внутрибанковский перевод с договора,5427
7,a056e194d3,2024-10-29,2024-10-29 18:00:00,-900.00,₽,-900.00,₽,Оплата в PE SPIRKOV IVAN SERGEEVICH г Москва RU,5037
8,b6dff213a1,2024-10-29,2024-10-29 17:48:00,-79.00,₽,-79.00,₽,Оплата в IP Shporunenko Y A g Moskva RUS,1665
9,9c5eb3ab85,2024-10-29,2024-10-29 15:20:00,-976.33,₽,-976.33,₽,Оплата в PYATEROCHKA 450 MOSCOW RUS,1665


In [37]:
# missing_recrods['Время операции'].astype(str)

In [38]:
# missing_recrods['Дата и время операции'] = missing_recrods['Дата и время операции'].dt.strftime('%Y-%m-%d %H:%M')
# missing_recrods['Дата списания'] = missing_recrods['Дата списания'].dt.strftime('%Y-%m-%d %H:%M')
missing_recrods['Дата операции'] = missing_recrods['Дата операции'].astype(str)
missing_recrods['Время операции'] = missing_recrods['Время операции'].dt.strftime('%Y-%m-%d %H:%M')


gsw.insert_df(missing_recrods, sheet_id, tbank_main_spreadsheet_name)

/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_3206/3182866917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_recrods['Дата операции'] = missing_recrods['Дата операции'].astype(str)
/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_3206/3182866917.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_recrods['Время операции'] = missing_recrods['Время операции'].dt.strftime('%Y-%m-%d %H:%M')
